In [76]:
import pandas as pd
import numpy as np
from pyplink import PyPlink
import statsmodels.api as sm

In [12]:
expression = pd.read_table("~/Downloads/180A/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt", delim_whitespace=True)
expression.set_index('TargetID', inplace=True)
expression.index = expression.index.str.replace("\.\d+", "", regex=True)
annotation = pd.read_csv("~/Downloads/180A/gene_annot.txt", sep="\t")
annotation.set_index('SYM', inplace=True)

In [13]:
expression.shape[0]

23722

In [58]:
'ENSG00000183696' in annotation.index

True

In [47]:
annotation

,ID,CHR,START,STOP,TYPE
SYM,,,,,
ENSG00000223972,DDX11L1,1,11868,13052,transcribed_unprocessed_pseudogene
ENSG00000186092,OR4F5,1,69090,70008,protein_coding
ENSG00000177757,FAM87B,1,817370,819834,lincRNA
ENSG00000225880,LINC00115,1,826205,827522,lincRNA
ENSG00000228794,LINC01128,1,827607,859446,processed_transcript
...,...,...,...,...,...
ENSG00000100299,ARSA,22,50622753,50628179,protein_coding
ENSG00000251322,SHANK3,22,50674414,50733298,protein_coding
ENSG00000100312,ACR,22,50738195,50745334,protein_coding


In [30]:
expression

,Gene_Symbol,Chr,Coord,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
TargetID,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931,ENSG00000152931.6,5,59783540,0.101858,0.078110,0.048981,0.118597,0.004035,0.010925,-0.000901,...,0.088601,0.240010,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
ENSG00000183696,ENSG00000183696.9,7,48128225,8.183805,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,...,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
ENSG00000139269,ENSG00000139269.2,12,57846106,1.199910,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,...,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954
ENSG00000169129,ENSG00000169129.8,10,116164515,0.831940,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
ENSG00000134602,ENSG00000134602.11,X,131157293,27.646422,24.395572,16.445374,24.806650,25.113349,19.233988,27.881116,...,25.079490,28.725528,24.450520,27.264069,26.912814,29.509210,26.462331,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472,ENSG00000235472.1,13,29172970,31.582832,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,...,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,30.598840,33.516674,32.017940,38.341888
ENSG00000114423,ENSG00000114423.14,3,105588396,14.054749,14.477899,11.584425,12.637956,12.015089,13.750655,15.690696,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117
ENSG00000243312,ENSG00000243312.2,4,87791344,1.112114,0.831797,0.253228,0.271568,0.486086,1.362640,1.543528,...,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.857330,0.703369


In [28]:
expression.index.str.replace("\.\d+", "", regex=True)

Index(['ENSG00000152931', 'ENSG00000183696', 'ENSG00000139269',
       'ENSG00000169129', 'ENSG00000134602', 'ENSG00000136237',
       'ENSG00000259425', 'ENSG00000242284', 'ENSG00000235027',
       'ENSG00000228169',
       ...
       'ENSG00000087095', 'ENSG00000261559', 'ENSG00000162144',
       'ENSG00000129473', 'ENSG00000261205', 'ENSG00000235472',
       'ENSG00000114423', 'ENSG00000243312', 'ENSG00000257337',
       'ENSG00000177494'],
      dtype='object', name='TargetID', length=23722)

In [35]:
expression.loc['ENSG00000152931'].values[3:]

array([0.101857770468582, 0.0781095249582053, 0.0489806714207954,
       0.118597118618172, 0.0040349636761474, 0.0109253485646219,
       -0.0009011975539238, -0.0067064609957826, 0.0988630165419412,
       0.0452846575848615, 0.0769367414340106, 0.246898644588526,
       0.0885540460643121, 0.0519733855651512, 0.0960895886763577,
       0.0638097933382033, 0.110985673181817, 0.100075574648624,
       0.146880535433817, 0.0467660507583092, 0.0629709774785819,
       0.150544722548943, 0.04199891708349, 0.0751741396297664,
       0.212984611529124, 0.0393149802293333, 0.0885200655979845,
       0.0478412670285812, 0.114000432145881, 0.0345676282300169,
       0.0394868546499791, 0.127594045094677, 0.125433082953702,
       0.0623176347234739, 0.0202659432269564, 0.0570715333409336,
       0.0084493746954648, 0.0606173017394201, 0.0448436257057132,
       0.0991055713166859, 0.0926258586401873, 0.100143289487024,
       -0.0162577578606756, 0.0418135018387938, 0.0189323018488825,
      

In [23]:
bim = PyPlink("/Users/johndriscoll/Downloads/180A/LDREF/1000G.EUR.1").get_bim()

,chrom,pos,cm,a1,a2
snp,,,,,
rs3094315,1,752566,0.488776,G,A
rs3131972,1,752721,0.488868,A,G
rs3131969,1,754182,0.489734,A,G
rs1048488,1,760912,0.492507,C,T
rs3115850,1,761147,0.492605,T,C
...,...,...,...,...,...
rs4926502,1,249209140,293.389210,A,G
rs6704311,1,249210707,293.390550,A,G
rs34013644,1,249211350,293.391090,C,T


In [89]:
expression[expression.columns[4:].values]

,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,HG00105,HG00106,HG00108,...,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20816,NA20819,NA20826,NA20828
TargetID,,,,,,,,,,,,,,,,,,,,,
ENSG00000152931,0.078110,0.048981,0.118597,0.004035,0.010925,-0.000901,-0.006706,0.098863,0.045285,0.076937,...,0.088601,0.240010,0.137175,0.148494,0.038643,0.088509,0.029204,0.024423,0.044816,0.139186
ENSG00000183696,5.686911,2.434653,3.830894,6.612288,4.709646,7.348876,8.180940,8.721889,8.169477,4.090099,...,13.428205,6.094500,12.536000,2.217262,3.573394,7.583364,4.052882,1.570378,4.900372,6.737308
ENSG00000139269,1.573572,0.521616,1.447225,3.565791,1.982681,0.675305,3.817395,2.561376,1.231049,1.269440,...,3.225880,1.996067,2.854923,2.267343,1.331201,2.187895,1.004250,3.003316,1.984362,1.684954
ENSG00000169129,0.069778,0.931086,0.620941,1.660668,0.570481,1.259393,0.734784,1.479124,1.548653,1.010198,...,1.023381,1.127852,0.774409,1.495854,0.895342,1.513521,0.826377,1.021201,0.952502,0.740565
ENSG00000134602,24.395572,16.445374,24.806650,25.113349,19.233988,27.881116,27.194117,28.579857,27.226416,25.384714,...,25.079490,28.725528,24.450520,27.264069,26.912814,29.509210,26.462331,25.624009,25.707741,22.824957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000235472,34.071123,19.394365,37.523721,33.430473,37.844966,35.843048,40.455377,32.287466,37.636318,29.742259,...,47.905182,33.224977,39.827675,27.096811,34.686342,37.386766,30.598840,33.516674,32.017940,38.341888
ENSG00000114423,14.477899,11.584425,12.637956,12.015089,13.750655,15.690696,11.798242,15.461657,18.126433,9.870305,...,11.723462,9.900372,10.473115,13.433413,15.832594,19.216176,10.213739,14.563192,15.637732,8.357117
ENSG00000243312,0.831797,0.253228,0.271568,0.486086,1.362640,1.543528,0.535700,0.900239,0.929809,0.755062,...,1.168991,0.645389,0.819469,0.515448,0.463054,1.580658,0.701396,0.771233,0.857330,0.703369


In [80]:
np.savetxt("individuals_with_expression.txt", np.array([expression.columns[4:].values, expression.columns[4:].values]).T, fmt='%s')

In [82]:
for i in range(1,23):
    ! ~/Downloads/180A/plink2 -bfile /Users/johndriscoll/Downloads/180A/LDREF/1000G.EUR.{i} -keep individuals_with_expression.txt -make-bed -out /Users/johndriscoll/Downloads/180B/LDREF/1000G.EUR.{i}

PLINK v2.00a6 AVX2 (11 Oct 2023)               www.cog-genomics.org/plink/2.0/
(C) 2005-2023 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/johndriscoll/Downloads/180B/LDREF/1000G.EUR.1.log.
Options in effect:
  --bfile /Users/johndriscoll/Downloads/180A/LDREF/1000G.EUR.1
  --keep individuals_with_expression.txt
  --make-bed
  --out /Users/johndriscoll/Downloads/180B/LDREF/1000G.EUR.1

Start time: Sun Jan 28 15:27:34 2024
16384 MiB RAM detected; reserving 8192 MiB for main workspace.
Using up to 8 compute threads.
489 samples (0 females, 0 males, 489 ambiguous; 489 founders) loaded from
/Users/johndriscoll/Downloads/180A/LDREF/1000G.EUR.1.fam.
98642 variants loaded from
/Users/johndriscoll/Downloads/180A/LDREF/1000G.EUR.1.bim.
Note: No phenotype data present.
--keep: 343 samples remaining.
343 samples (0 females, 0 males, 343 ambiguous; 343 founders) remaining after
main filters.
Writing /Users/johndriscoll/Downloads/180B/LDREF/1000G.EUR.1.fam ... do

In [83]:
pyplinkdict = {i: PyPlink("/Users/johndriscoll/Downloads/180B/LDREF/1000G.EUR.{}".format(i)) for i in range(1,23)}

In [86]:
pyplinkdict

{1: PyPlink(343 samples; 98,642 markers),
 2: PyPlink(343 samples; 99,735 markers),
 3: PyPlink(343 samples; 83,036 markers),
 4: PyPlink(343 samples; 74,924 markers),
 5: PyPlink(343 samples; 75,164 markers),
 6: PyPlink(343 samples; 75,358 markers),
 7: PyPlink(343 samples; 66,171 markers),
 8: PyPlink(343 samples; 64,975 markers),
 9: PyPlink(343 samples; 55,464 markers),
 10: PyPlink(343 samples; 64,067 markers),
 11: PyPlink(343 samples; 60,977 markers),
 12: PyPlink(343 samples; 58,543 markers),
 13: PyPlink(343 samples; 45,546 markers),
 14: PyPlink(343 samples; 39,484 markers),
 15: PyPlink(343 samples; 35,839 markers),
 16: PyPlink(343 samples; 36,526 markers),
 17: PyPlink(343 samples; 32,218 markers),
 18: PyPlink(343 samples; 35,513 markers),
 19: PyPlink(343 samples; 22,509 markers),
 20: PyPlink(343 samples; 31,101 markers),
 21: PyPlink(343 samples; 17,040 markers),
 22: PyPlink(343 samples; 17,489 markers)}

In [94]:
expression = expression[pyplinkdict[1].get_fam()['fid'].values]

In [103]:
for symbol in expression.index:
    if symbol not in annotation.index:
        continue
    gene_name, chromosome, gene_start = annotation.loc[symbol][["ID", "CHR", "START"]].values
    cis_start, cis_end = gene_start - 500000, gene_start + 500000
    if cis_start < 0:
        cis_start = 0
    max_stop = max(annotation["STOP"])
    if cis_end > max_stop:
        cis_end = max_stop

    curr_bim = pyplinkdict[chromosome].get_bim()
    curr_expression = expression.loc[symbol].values

    #select snps within gene body
    snp_subset = curr_bim[(curr_bim['pos'] > cis_start) & (curr_bim['pos'] < cis_end)].reset_index()

    #display(snp_subset)
    summary_matrix = []

    N = pyplinkdict[chromosome].get_nb_samples()
    snp_subset['N'] = [N] * snp_subset.shape[0]
    snp_subset['type'] = ['cc'] * snp_subset.shape[0]

    genotypes = []
    mafs = []
    for _, [snp, a1] in snp_subset[['snp', 'a1']].iterrows():
        #print(snp)
        alleles = pyplinkdict[chromosome].get_geno_marker(snp)
        unique_cts = np.unique(alleles, return_counts=True)[1]
        maf = min(unique_cts/sum(unique_cts))
        genotypes += [alleles]
        mafs += [maf]
    snp_subset['genotypes'] = genotypes
    snp_subset['MAF'] = mafs
    
    #print(len(curr_expression), len(genotypes[0]))
    snp_subset['results'] = snp_subset['genotypes'].apply(lambda x: sm.OLS(curr_expression, x).fit())
    
    snp_subset['pval'] = snp_subset['results'].apply(lambda x: x.pvalues[0])
    snp_subset['bse'] = snp_subset['results'].apply(lambda x: x.bse[0])
    snp_subset['varbeta'] = snp_subset['bse']**2
    snp_subset['effect_size'] = np.e**snp_subset['results'].apply(lambda x: x.params[0])
    snp_subset['beta'] = snp_subset['results'].apply(lambda x: x.params[0])

    snp_subset[['beta', 'varbeta', 'snp', 'pos', 'type', 'N', 'MAF', 'pval']].to_csv("eQTL_subsets/{}".format(symbol), index=False)
    #print("{}".format(symbol))
    display(snp_subset)
    break

,snp,chrom,pos,cm,a1,a2,N,type,genotypes,MAF,results,pval,bse,varbeta,effect_size,beta
0,rs16878315,5,59989842,71.904406,G,A,343,cc,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.064140,<statsmodels.regression.linear_model.Regressio...,3.675664e-03,0.019960,0.000398,1.060119,0.058381
1,rs7712043,5,59990319,71.904407,C,T,343,cc,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.064140,<statsmodels.regression.linear_model.Regressio...,3.675664e-03,0.019960,0.000398,1.060119,0.058381
2,rs6896680,5,59991531,71.904412,C,T,343,cc,"[1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, ...",0.084548,<statsmodels.regression.linear_model.Regressio...,1.175042e-28,0.004807,0.000023,1.060377,0.058625
3,rs12187325,5,59992279,71.904415,C,T,343,cc,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...",0.008746,<statsmodels.regression.linear_model.Regressio...,2.706489e-10,0.010842,0.000118,1.073116,0.070566
4,rs12517207,5,59993268,71.904446,C,T,343,cc,"[1, 1, 1, 2, 2, 0, 1, 0, 0, 1, 0, 2, 1, 1, 1, ...",0.180758,<statsmodels.regression.linear_model.Regressio...,2.571307e-32,0.003790,0.000014,1.051176,0.049909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,rs16893687,5,60982841,72.606148,A,G,343,cc,"[0, 0, 2, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, ...",0.107872,<statsmodels.regression.linear_model.Regressio...,3.999285e-29,0.004546,0.000021,1.057623,0.056024
363,rs6882777,5,60983430,72.606219,T,C,343,cc,"[1, 2, 0, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2, ...",0.177843,<statsmodels.regression.linear_model.Regressio...,5.826247e-31,0.003775,0.000014,1.049552,0.048363
364,rs377667,5,60984261,72.606319,C,T,343,cc,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...",0.011662,<statsmodels.regression.linear_model.Regressio...,2.382654e-11,0.009158,0.000084,1.065324,0.063279
365,rs381182,5,60986143,72.606544,C,T,343,cc,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...",0.011662,<statsmodels.regression.linear_model.Regressio...,2.382654e-11,0.009158,0.000084,1.065324,0.063279
